In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask-ngrok
!pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-qdbx_2i7
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-qdbx_2i7
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3082494 sha256=c92e3d70e496d1acee31a617d0f75e4f94b093531d688aec39e9ffaf091e420d
  Stored in directory: /tmp/pip-ephem-wheel-cache-wawwhpox/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 26kB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 512kB 28.0MB/s 
     |████████████████████████████████| 3.8MB 28.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=64bfbca752b969739b8c0f2b9e2813e292b9e6f76b0ba452ffceadb467978538
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
import numpy as np
from numpy import argmax

In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-ver1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
#get word embedding libary
import fasttext.util
print('Creating fastText...')
word2vec = fasttext.load_model('/content/drive/MyDrive/fast_text/cc.vi.300.bin')
print('fastText created!')

Creating fastText...
fastText created!


In [ ]:
!pip install vncorenlp

     |████████████████████████████████| 2.7MB 5.2MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=2967bfbbaae8c877642f2fea583f7a2b45e56fad44833dd28674233ae5c35769
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
# get word embedding
from vncorenlp import VnCoreNLP
print('Creating token word...')
annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
print('token word created')

Creating token word...
token word created


In [ ]:
# aphabet tag library
tags = {'pad': 0, 'LOCATION_HOMENUMBER': 1, 'LOCATION_STREET': 2, 'LOCATION_WARD': 3, 'LOCATION_DISTRICT': 4, 'LOCATION_PROVINCE': 5, 'LOCATION_COUNTRY': 6, 'LOCATION_POSTCODE': 7, 'LOCATION_SPECIAL': 8, 'LOCATION_NER': 9, 'OBJ': 10, 'OBJ_FEATURE': 11, 'PRE': 12, 'UNKNOW': 13 }
aphabet_tag = []
for key in tags:
  aphabet_tag.append(key)
# aphabet_tag = ['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW', 'LOCATION_SPECIAL']
print(aphabet_tag)

['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'LOCATION_SPECIAL', 'LOCATION_NER', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW']


In [ ]:
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>2:
        return u'<hem>'

    if re.match(r"^[0-9]{2}0{3}$", word):
        return u'<postcode>'

    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    word = word.replace("_"," ")
    return word

def get_word_embbeding(sen_words,max_length=32,embedd_dim = 300):
  X = np.empty([1, max_length, embedd_dim])
  unknown_embedd = np.random.uniform(-0.01, 0.01, [1, 300])
  length = len(sen_words)
  for i in range(length):
    try:
      vec = word2vec.get_word_vector(map_number_and_punct(sen_words[i])).tolist()
    except:
      vec = unknown_embedd
    X[0, i, :] = vec
  X[0,length:] = np.zeros([1, embedd_dim])
  return np.array(X)
# sen_words_embedd = get_word_embbeding(sen_words[0],max_length=25,embedd_dim = 300)
# print(sen_words_embedd.shape)

In [ ]:
#get char embedding
def read_char_vocab(file_path):
  char_vocab = []
  for line in open(file_path, encoding='utf-8'):
    char_vocab.append(line.splitlines()[0])      
  return char_vocab

def char_to_int(char_vocab_data):
  dic = {}
  index = -1
  for char in char_vocab_data:
    try:
      dic[char]
    except:
      index = index + 1
      dic[char]=index
  return dic
 
def get_char_encode(sentence,max_length_of_a_sentence,max_length_of_a_word):
  char_vocab_data = read_char_vocab("/content/drive/MyDrive/fast_text/VISCII_short.txt")
  dic = char_to_int(char_vocab_data)
  # sentence  words is a sentence | ['i','am','an']
  sentence_encoded = np.zeros([max_length_of_a_sentence,max_length_of_a_word]) # 25*25
  for j in range(len(sentence)):  
    word = sentence[j].lower()
    # integer_encoded = [char_to_int[char] for char in word]
    word_encoded = np.zeros(max_length_of_a_word)
    for k in range(len(word)):
      char = word[k]
      try:
        word_encoded[k]= dic[char]
      except:
        print("error : " + str(char)+" "+str(word)+" "+str(len(word)))
        word_encoded[k]= dic['[unk]']
    # sentence encoded
    sentence_encoded[j] = word_encoded
  return sentence_encoded

def get_charEmbedd_form_encode(charEnocde):
  LEN_OF_VOCAB = 137
  shape = charEnocde.shape
  char_embedd = np.zeros([shape[0],shape[1],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      char_int = charEnocde[i,j]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,:] = onehot
  return char_embedd

# return onehot_encoded_of_a_sentence

In [ ]:
!unzip /content/drive/MyDrive/ngrok/ngrok-stable-linux-amd64.zip

Archive:  /content/drive/MyDrive/ngrok/ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!./ngrok authtoken 1ow8g2aFZ4wILzg2dqHin5bUy5t_34f12x5zn89tGiLLCi4Wy

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# query = "số nhà 103 - A12"
# # query = "tìm các nơi tổ chức tang lễ đang mở cửa xung quanh khu này"
# try:
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# except:
#   print('Creating token word...')
#   annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
#   print('token word created')
#   sen_words = annotator.tokenize(query)[0]
#   num_of_word = len(sen_words)
# print(sen_words)

['số', 'nhà', '103', '-', 'A12']


In [ ]:
# hàm tiền xử lý câu đầu vào
# bỏ các dấu đặc biệt : 
special_char = [",","_"]
# bỏ các từ đặc biệt: quận, huyện, tỉnh, thành phố, 
fail_text = ["ward", "xã", "phường", "thị trấn", "quận", "huyện","tỉnh","district","thị xã","thành phố","province"]
def process_word(word):
  for char in special_char:
    if char in word:
      word = word.replace(char," ")
  for text in fail_text:
    if text in word:
      word = word.replace(text," ")
  return word.lower()

In [ ]:
# query = "tìm kiếm khách sạn 5 sao xung quanh chỗ này"
# query = "sân bay đang mở cửa xung quanh 144 lô 14a trung hòa cầu giấy hà nội việt nam"
# query = " nghĩa đô cầu giấy hà nội việt nam 10000"
# query = "khách sạn 5 sao gần số 6 ngõ 5 cầu giấy"
# query = "293 Trần Đại Nghĩa quận Hai Bà Trưng Hà Nội"
# query = "nguyễn thị định trung hoà cầu giấy hà nội việt nam 10000"
# query = "144 xuân thủy cầu giấy hà nội"
query = "tìm các nhà thương tại phở cuốn hương mai vũ phạm hàm"
try:
  sen_words = annotator.tokenize(query)[0]
  num_of_word = len(sen_words)
except:
  print('Creating token word...')
  annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
  print('token word created')
  sen_words = annotator.tokenize(query)[0]
  num_of_word = len(sen_words)
print(sen_words)

for i in range(len(sen_words)):
  sen_words[i] = process_word(sen_words[i])

# get word embedding
sen_words_embedd = get_word_embbeding(sen_words,max_length=42,embedd_dim = 300)
    
# get char embedding
char_encode = get_char_encode(sen_words,42,32)

char_embedd = get_charEmbedd_form_encode(char_encode)
char_embedd = char_embedd.reshape(1,char_embedd.shape[0], char_embedd.shape[1],char_embedd.shape[2])

# predict
output = m.predict([char_embedd,sen_words_embedd])

# return [1,2,3,4,5,5,5,5,5,5,5,5,5,5,5] or convert to encode tag
tag_encode = []
for i in range(42): 
  tag_encode.append(argmax(output[0,i,:]))

# return [street, homenumber,.....] or only get [num_of_word] first element=> convert to tag
tag_result=[]
for i in range(num_of_word):
  tag_result.append(aphabet_tag[tag_encode[i]])
  arr_tag_result=np.array(tag_result)

# return {'nhaf_hàng': OBJ,....} 
result={}
for i in range(num_of_word):
  result[sen_words[i]] = str(arr_tag_result[i])

print(result)
#==========================
dic_result = {}
for word in result:
  tag = result[word]
  try:
    dic_result[tag] += " " + word
  except:
    dic_result[tag] = word
print(dic_result)

['tìm', 'các', 'nhà_thương', 'tại', 'phở', 'cuốn', 'hương', 'mai', 'vũ', 'phạm', 'hàm']
error :   nhà thương 10
{'tìm': 'UNKNOW', 'các': 'UNKNOW', 'nhà thương': 'OBJ', 'tại': 'OBJ_FEATURE', 'phở': 'OBJ_FEATURE', 'cuốn': 'OBJ_FEATURE', 'hương': 'LOCATION_STREET', 'mai': 'OBJ_FEATURE', 'vũ': 'pad', 'phạm': 'pad', 'hàm': 'LOCATION_PROVINCE'}
{'UNKNOW': 'tìm các', 'OBJ': 'nhà thương', 'OBJ_FEATURE': 'tại phở cuốn mai', 'LOCATION_STREET': 'hương', 'pad': 'vũ phạm', 'LOCATION_PROVINCE': 'hàm'}


In [ ]:
# get word embedding
from vncorenlp import VnCoreNLP
print('Creating token word...')
annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
print('token word created')

Creating token word...
token word created


In [ ]:
import requests
#from flask import Flask, render_template, jsonify
from flask import Flask, render_template, abort, request, jsonify, json,Response
from flask import request, redirect, url_for
import codecs
import gensim
from distutils.version import LooseVersion, StrictVersion
import json
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)
app.config['JSON_AS_ASCII'] = False

global word2vec_model

@app.route('/')
def summary():
  if request.method == "GET":
    query = request.values['query'] or ''
    query = str(query)
    print( 'query = ' + query )
    
    # token word
    try:
      sen_words = annotator.tokenize(query)[0]
      num_of_word = len(sen_words)
    except:
      print('Creating token word...')
      annotator = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
      print('token word created')
      sen_words = annotator.tokenize(query)[0]
      num_of_word = len(sen_words)
    print(sen_words)
        
    # get word embedding
    sen_words_embedd = get_word_embbeding(sen_words,max_length=42,embedd_dim = 300)
    print("sen_words_embedd done!")
    # get char embedding
    char_encode = get_char_encode(sen_words,42,32)

    char_embedd = get_charEmbedd_form_encode(char_encode)
    char_embedd = char_embedd.reshape(1,char_embedd.shape[0], char_embedd.shape[1],char_embedd.shape[2])
    print("char_embedd done!")
    # predict
    output = m.predict([char_embedd,sen_words_embedd])
    print("output done!")
    # return [1,2,3,4,5,5,5,5,5,5,5,5,5,5,5] or convert to encode tag
    tag_encode = []
    for i in range(42): 
      tag_encode.append(argmax(output[0,i,:]))

    # return [street, homenumber,.....] or only get [num_of_word] first element=> convert to tag
    tag_result=[]
    for i in range(num_of_word):
      tag_result.append(aphabet_tag[tag_encode[i]])
      arr_tag_result=np.array(tag_result)

    # return {'nhaf_hàng': OBJ,....} 
    result={}
    for i in range(num_of_word):
      result[sen_words[i]] = str(arr_tag_result[i])

    print(result)
    #==========================
    dic_result = {}
    for word in result:
      tag = result[word]
      try:
        dic_result[tag] += ", " + word
      except:
        dic_result[tag] = word
    print(dic_result)

    return jsonify(sen_words=sen_words, tags=tag_result, output=dic_result)
if __name__ == "__main__":
  import os
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://96763f8082c6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
query = tìm kiếm khách sạn quanh đây
Creating token word...


127.0.0.1 - - [24/Apr/2021 01:31:02] "GET /?query=tìm%20kiếm%20khách%20sạn%20quanh%20đây HTTP/1.1" 200 -


token word created
['tìm_kiếm', 'khách_sạn', 'quanh', 'đây']
sen_words_embedd done!
char_embedd done!
output done!
{'tìm_kiếm': 'UNKNOW', 'khách_sạn': 'OBJ', 'quanh': 'PRE', 'đây': 'LOCATION_SPECIAL'}
{'UNKNOW': 'tìm_kiếm', 'OBJ': 'khách_sạn', 'PRE': 'quanh', 'LOCATION_SPECIAL': 'đây'}


...

token word created
['tìm_kiếm', 'các', 'trung_tâm', 'nghệ_thuật', 'đang', 'mở_cửa', 'ở', 'quán', 'bi_a']
sen_words_embedd done!
char_embedd done!
output done!
{'tìm_kiếm': 'UNKNOW', 'các': 'UNKNOW', 'trung_tâm': 'OBJ', 'nghệ_thuật': 'OBJ', 'đang': 'OBJ_FEATURE', 'mở_cửa': 'OBJ_FEATURE', 'ở': 'PRE', 'quán': 'LOCATION_SPECIAL', 'bi_a': 'pad'}
{'UNKNOW': 'tìm_kiếm, các', 'OBJ': 'trung_tâm, nghệ_thuật', 'OBJ_FEATURE': 'đang, mở_cửa', 'PRE': 'ở', 'LOCATION_SPECIAL': 'quán', 'pad': 'bi_a'}
query = tìm kiếm các văn phòng giao hàng đang mở cửa ở quán bi a
Creating token word...


127.0.0.1 - - [24/Apr/2021 02:18:09] "GET /?query=tìm%20kiếm%20các%20văn%20phòng%20giao%20hàng%20đang%20mở%20cửa%20ở%20quán%20bi%20a HTTP/1.1" 200 -


token word created
['tìm_kiếm', 'các', 'văn_phòng_giao_hàng', 'đang', 'mở_cửa', 'ở', 'quán', 'bi_a']
sen_words_embedd done!
char_embedd done!
output done!
{'tìm_kiếm': 'UNKNOW', 'các': 'UNKNOW', 'văn_phòng_giao_hàng': 'OBJ', 'đang': 'OBJ_FEATURE', 'mở_cửa': 'OBJ_FEATURE', 'ở': 'PRE', 'quán': 'LOCATION_SPECIAL', 'bi_a': 'pad'}
{'UNKNOW': 'tìm_kiếm, các', 'OBJ': 'văn_phòng_giao_hàng', 'OBJ_FEATURE': 'đang, mở_cửa', 'PRE': 'ở', 'LOCATION_SPECIAL': 'quán', 'pad': 'bi_a'}
query = tìm kiếm các cửa hàng giày dép đang mở cửa ở quán bi a
Creating token word...


127.0.0.1 - - [24/Apr/2021 02:22:12] "GET /?query=tìm%20kiếm%20các%20cửa%20hàng%20giày%20dép%20đang%20mở%20cửa%20ở%20quán%20bi%20a HTTP/1.1" 200 -


token word created
['tìm_kiếm', 'các', 'cửa_hàng_giày_dép', 'đang', 'mở_cửa', 'ở', 'quán', 'bi_a']
sen_words_embedd done!
char_embedd done!
output done!
{'tìm_kiếm': 'UNKNOW', 'các': 'UNKNOW', 'cửa_hàng_giày_dép': 'OBJ', 'đang': 'OBJ_FEATURE', 'mở_cửa': 'OBJ_FEATURE', 'ở': 'PRE', 'quán': 'LOCATION_SPECIAL', 'bi_a': 'pad'}
{'UNKNOW': 'tìm_kiếm, các', 'OBJ': 'cửa_hàng_giày_dép', 'OBJ_FEATURE': 'đang, mở_cửa', 'PRE': 'ở', 'LOCATION_SPECIAL': 'quán', 'pad': 'bi_a'}
